In [41]:
from google.cloud import storage
from firebase_admin import firestore, initialize_app

# Establish a connection to the Google Cloud Storage and Firestore
storage_client = storage.Client()
bucket = storage_client.bucket('website-content12345')
initialize_app()
db = firestore.client()

In [2]:
from datetime import datetime
# Okay let's insert some data into Firestore
doc_ref = db.collection('feed').document('content-log')

timestamp = datetime.now()
timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")

data = {
    timestamp: {
        'title': 'Test Blog',
        'location': 'blogs/test-blog.md',
    }
}

# Upload it
doc_ref.set(data)

update_time {
  seconds: 1720282478
  nanos: 194105000
}

In [4]:
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()

# Sort by key (timestamp) desc
data = dict(sorted(data.items(), key=lambda item: item[0], reverse=True))
data

{'2024-07-06 17:14:38': {'location': 'blogs/test-blog.md'}}

In [9]:
for key, value in data.items():
    blob = bucket.blob(value['location'])
    md = blob.download_as_string().decode('utf-8')

# We can capture the section between --- and --- and use it as metadata
metadata = md.split('---')[1]
description = {}
for line in metadata.split('\n'):
    if line:
        split_line = line.split(':')
        key = split_line[0].strip()
        # Remove the quotes
        value = ":".join(split_line[1:]).strip()[1:-1]
        description[key.strip()] = value

description

{'title': 'Test Post',
 'author': 'Ed',
 'date': '2024-07-06',
 'tags': '"coding", "python"',
 'type': 'blog',
 'description': 'This is a blog post about coding in Python.',
 'thumbnail': 'images/awesome-blog-post-thumbnail.jpg',
 'og_title': 'Awesome Blog Post',
 'og_description': 'An amazing blog post about coding in Python.',
 'og_image': 'images/awesome-blog-post-og.jpg'}

In [2]:
# Find md files in test-dir
import os
md_files = []
images = []
for root, dirs, files in os.walk('test-dir'):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))
        elif file.endswith('.png') :
            images.append(os.path.join(root, file))

print(md_files)
print("-"*10)
print(images)
print("-"*10)
print(len(md_files))
print("-"*10)
print(len(images))

['test-dir/2024/January/books.md', 'test-dir/2024/February/hip.md', 'test-dir/2024/May/discord.md', 'test-dir/2024/April/internet.md', 'test-dir/2024/March/horses.md', 'test-dir/2023/December/depressed.md', 'test-dir/2023/December/charlmes.md', 'test-dir/2023/July/burnout.md', 'test-dir/2023/July/onrss.md', 'test-dir/2023/July/dustydrawers.md', 'test-dir/2023/August/stress.md', 'test-dir/2023/August/prank.md', 'test-dir/2023/February/stumbleupon.md', 'test-dir/2023/June/alcoholism.md', 'test-dir/2023/June/oldwebsites.md', 'test-dir/2023/April/bananas.md']
----------
['test-dir/blog/2024/phone.png', 'test-dir/blog/2024/horse.png', 'test-dir/blog/2024/skateboard.png', 'test-dir/blog/2024/discord.png', 'test-dir/blog/2023/rss.png', 'test-dir/blog/2023/stumbleUpon.png', 'test-dir/blog/2023/depression.png', 'test-dir/blog/2023/duck.png', 'test-dir/blog/2023/banana.png', 'test-dir/blog/2023/outside.png', 'test-dir/blog/2023/computer.png', 'test-dir/blog/2023/stress.png', 'test-dir/blog/2023/

In [4]:
# Now we need to add metadata to the md files and upload them to the bucket
# The thumbnails can be compressed jpgs of the images
# Oh we also need to link the images in the md files to the image files
related_images = {}
# Go through our markdown files - md file is the key and any images in the doc are the values
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()
    images = []
    for line in md.split('\n'):
        if '![' in line:
            images.append(line.split('(')[1].split(')')[0])
    related_images[md_file] = {
        'images': images
    }

print(related_images)

{'test-dir/2024/January/books.md': {'images': []}, 'test-dir/2024/February/hip.md': {'images': ['/images/blog/2024/skateboard.png']}, 'test-dir/2024/May/discord.md': {'images': ['/images/blog/2024/discord.png']}, 'test-dir/2024/April/internet.md': {'images': ['/images/blog/2024/phone.png']}, 'test-dir/2024/March/horses.md': {'images': ['/images/blog/2024/horse.png']}, 'test-dir/2023/December/depressed.md': {'images': ['/images/blog/2023/depression.png']}, 'test-dir/2023/December/charlmes.md': {'images': ['/images/blog/2023/duck.png']}, 'test-dir/2023/July/burnout.md': {'images': ['/images/blog/2023/burnout.png']}, 'test-dir/2023/July/onrss.md': {'images': ['/images/blog/2023/rss.png']}, 'test-dir/2023/July/dustydrawers.md': {'images': ['/images/blog/2023/hobbies.png', '/images/blog/2023/outside.png']}, 'test-dir/2023/August/stress.md': {'images': ['/images/blog/2023/stress.png']}, 'test-dir/2023/August/prank.md': {'images': ['/images/blog/2023/computer.png']}, 'test-dir/2023/February/s

In [7]:
from bs4 import BeautifulSoup
metadata = {}
# Cool now we can get the date these were published and the titles
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()
    # First line is date, rm the # and strip
    date = md.split('\n')[0].strip('#').strip().replace("/", "-")
    # Second line is title, rm the # and strip
    title = md.split('\n')[1].strip('#').strip()

    # We can open the corresponding html file to get the og tags
    html_file = md_file.replace('.md', '.html')
    with open(html_file, 'r') as f:
        html = f.read()

    # Use bs
    soup = BeautifulSoup(html, 'html.parser')
    # Get og tags
    og_tags = {}
    for tag in soup.find_all('meta'):
        if tag.get('property') and tag.get('content'):
            og_tags[tag.get('property').replace(":", "_")] = tag.get('content')

    # Author is always Ed
    author = 'Ed'

    # We need some tags
    # We can set some default tags since this is my peronsal blog about silly things
    tags = ['Silly', 'Personal', 'Lifestyle']

    # Type is blog
    _type = 'blog'

    # Now we just need a thumbnail which we'll copy the first image name, add _thumbnail, compress to 64x64 and make a .jpg
    try:
        thumbnail = related_images[md_file]['images'][0].replace('.png', '_thumbnail.jpg')
    except:
        thumbnail = None

    # Now set all this data
    data = {
        'date': date,
        'title': title,
        'author': author,
        'tags': tags,
        'type': _type,
        'thumbnail': thumbnail,
        'og_tags': og_tags
    }

    metadata[md_file] = data
    metadata[md_file]['images'] = related_images[md_file]['images']

metadata

{'test-dir/2024/January/books.md': {'date': '04-01-2024',
  'title': 'Best Books of 2023',
  'author': 'Ed',
  'tags': ['Silly', 'Personal', 'Lifestyle'],
  'type': 'blog',
  'thumbnail': None,
  'og_tags': {'og_title': 'Best Books of 2023',
   'og_description': 'I started 2023 off strong with my reading, absolutely ploughing through books at a rate of one every other day, but this significantly slowed down in September ...',
   'og_type': 'article'},
  'images': []},
 'test-dir/2024/February/hip.md': {'date': '01-02-2024',
  'title': 'Why Do I Keep Injuring My Left Hip Specifically?',
  'author': 'Ed',
  'tags': ['Silly', 'Personal', 'Lifestyle'],
  'type': 'blog',
  'thumbnail': '/images/blog/2024/skateboard_thumbnail.jpg',
  'og_tags': {'og_title': 'Why Do I Keep Injuring My Left Hip Specifically?',
   'og_image': '/images/blog/2024/skateboard.png',
   'og_description': "When I was 30 years old I had a great idea. In spite of having never rollerskated in my life (apart from apparent

In [14]:
# Now we can insert this as metadata into our blog markdown and move them up to the root directory and also strip all the path from the image paths except filename
# Also images doesn't need to be included in the metadata it's just so we can track what goes in the bucket
# All our md files need the image paths setting to /assets/images/imagename.png
# With that in mind let's go ahead and do it
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()

    # Remove the first two lines as these are title/date which we'll render from the metadata
    md = "\n".join(md.split('\n')[2:])

    # Now strip any excess lines
    md = md.strip()

    # Replace the image paths
    for image in metadata[md_file]['images']:
        md = md.replace(image, f"/assets/images/{image.split('/')[-1]}")

    # Get the metadata
    data = metadata[md_file]

    # Now we need to update the md file with the metadata
    new_md = f"""---
date: {data['date']}
title: {data['title']}
author: {data['author']}
tags: {data['tags']}
type: {data['type']}
thumbnail: /assets/images/{data['thumbnail'].split('/')[-1] if data['thumbnail'] else ''}
og_title: {data['og_tags'].get('og_title', '')}
og_description: {data['og_tags'].get('og_description', '')}
og_image: {data['og_tags'].get('og_image', '')}
og_type: {data['og_tags'].get('og_type', '')}
---
{md}
"""

    if not os.path.exists('test-dir/blogs'):
        os.makedirs('test-dir/blogs')

    # Write this new md to test-dir/blogs
    with open(f"test-dir/blogs/{md_file.split('/')[-1]}", 'w') as f:
        f.write(new_md)

# Now let's copy all our images to test-dir/images
import shutil
if not os.path.exists('test-dir/images'):
    os.makedirs('test-dir/images')

for image in images:
    shutil.copy(os.path.join('test-dir', *image.split('/')[2:]), f"test-dir/images/{image.split('/')[-1]}")

In [16]:
images = []
for root, dirs, files in os.walk('test-dir'):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))
        elif file.endswith('.png') :
            images.append(os.path.join(root, file))

In [19]:

for image in images:
    try:
        shutil.copy(os.path.join('test-dir', *image.split('/')[1:]), f"test-dir/images/{image.split('/')[-1]}")
    except Exception as e:
        print(e)


'test-dir/images/banana.png' and 'test-dir/images/banana.png' are the same file


In [20]:
# Date format is incorrect it goes dd-mm-yyyy instead of yyyy-mm-dd
# Let's fix that
for root, dirs, files in os.walk('test-dir/blogs'):
    for file in files:
        with open(os.path.join(root, file), 'r') as f:
            md = f.read()
        date = md.split('date: ')[1].split('\n')[0]
        date = date.split('-')
        date = f"{date[2]}-{date[1]}-{date[0]}"
        md = md.replace(md.split('date: ')[1].split('\n')[0], date)
        with open(os.path.join(root, file), 'w') as f:
            f.write(md)


In [21]:
# Oh let's use PIL to make our thumbnail images
images = os.listdir('test-dir/images')
images

['phone.png',
 'horse.png',
 'rss.png',
 'stumbleUpon.png',
 'depression.png',
 'duck.png',
 'banana.png',
 'outside.png',
 'computer.png',
 'stress.png',
 'burnout.png',
 'hu.png',
 'skateboard.png',
 'stairs.png',
 'hobbies.png',
 'discord.png']

In [27]:
from PIL import Image
for image in images:
    img = Image.open(f"test-dir/images/{image}")
    img.thumbnail((256, 256))
    img = img.convert("RGB")
    img.save(f"test-dir/images/{image.replace('.png', '_thumbnail.jpg')}")

In [39]:
import re

pattern = r'(og_image: )(/images/blog/\d{4}/)'

# Our md files have incorrect image paths for og_image
for md_file in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', md_file), 'r') as f:
        md = f.read()

    # Regex replace the capture group with /assets/images/
    md = re.sub(pattern, r'\1/assets/images/', md)

    # Write it back
    with open(os.path.join('test-dir', 'blogs', md_file), 'w') as f:
        f.write(md)


In [42]:
# Cool now we need to write our firestore log
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()
data

{'2024-07-06 17:14:38': {'location': 'blogs/test-blog.md'}}

In [44]:
from datetime import datetime as dt
from datetime import timedelta

data = {}
# So we will overwrite this with all our blogs - we do actually have a last edit date in the original file metadata we can use
# But if it is prior to the actual date we will use the actual date at midnight
# So let's go ahead and do that
# First go through and get datetime objects for all the blogs
for year in ['2023', '2024']:
    for root, dirs, files in os.walk(os.path.join('test-dir', year)):
        for file in files:
            if file.endswith('.md'):
                with open(os.path.join(root, file), 'r') as f:
                    md = f.read()
                actual_date = md.split('\n')[0].strip('#').strip().replace("/", "-")
                # Parse the actual date
                actual_date = dt.strptime(actual_date, "%d-%m-%Y")
                # Set time to midnight
                actual_date = actual_date.replace(hour=0, minute=0, second=0, microsecond=0)

                # Now get the file edit datetime
                edited_time = dt.fromtimestamp(os.path.getmtime(os.path.join(root, file)))

                # Make sure that edited_time isn't more than 1 day ahead of actual_date
                if edited_time <= actual_date + timedelta(days=1):
                    actual_date = edited_time

                data[actual_date.strftime("%Y-%m-%d %H:%M:%S")] = {
                    'location': os.path.join('blogs', file),
                }

print(data)

{'2023-12-17 16:08:05': {'location': 'blogs/depressed.md'}, '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'}, '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'}, '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'}, '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'}, '2023-08-08 12:02:16': {'location': 'blogs/stress.md'}, '2023-08-22 15:15:32': {'location': 'blogs/prank.md'}, '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'}, '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'}, '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'}, '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'}, '2024-01-04 11:35:56': {'location': 'blogs/books.md'}, '2024-02-01 17:08:24': {'location': 'blogs/hip.md'}, '2024-05-06 09:38:07': {'location': 'blogs/discord.md'}, '2024-04-23 08:17:04': {'location': 'blogs/internet.md'}, '2024-03-14 00:00:00': {'location': 'blogs/horses.md'}}


In [45]:
# Banging job now let's just order it such that the most recent is first
data = dict(sorted(data.items(), key=lambda item: item[0], reverse=True))
data

{'2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'}}

In [46]:
# Slam it into firestore
feed.set(data)

update_time {
  seconds: 1720885488
  nanos: 98901000
}

In [51]:
# Go through and copy the og_description to the description field
for md_file in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', md_file), 'r') as f:
        md = f.read()

    # Get the og_description
    og_description = md.split('og_description: ')[1].split('\n')[0]

    # Add a description field under title
    title = md.split('title: ')[1].split('\n')[0]
    title_with_description = f"{title}\ndescription: {og_description}"
    md = md.replace(title, title_with_description)

    # Write it back
    with open(os.path.join('test-dir', 'blogs', md_file), 'w') as f:
        f.write(md)

In [53]:
# Rename our music files to add underscores in place of spaces
for root, dirs, files in os.walk('test-dir/music'):
    for file in files:
        if ' ' in file:
            os.rename(os.path.join(root, file), os.path.join(root, file.replace(' ', '_')))
        if '[' in file or ']' in file:
            os.rename(os.path.join(root, file), os.path.join(root, file.replace('[', '').replace(']', '')))

In [54]:
# Let's write a markdown file for our Planet Ed Album
metadata = {
    'date': '2005-01-01',
    'title': 'Planet Ed',
    'description': 'My first album, Planet Ed, was released in 2005. Nobody really cared. I was 12 going on 13.',
    'author': 'Ed',
    'tags': ['Music', 'Planet Ed'],
    'type': 'music',
    'thumbnail': '/assets/images/planet_ed.jpg',
    'og_title': 'Planet Ed',
    'og_description': 'My first album, Planet Ed, was released in 2005. Nobody really cared. I was 12 going on 13.',
    'og_image': '/assets/images/planet_ed.jpg',
    'og_type': 'music'
}

metadata_md = f"""---
date: {metadata['date']}
title: {metadata['title']}
description: {metadata['description']}
author: {metadata['author']}
tags: {metadata['tags']}
type: {metadata['type']}
thumbnail: {metadata['thumbnail']}
og_title: {metadata['og_title']}
og_description: {metadata['og_description']}
og_image: {metadata['og_image']}
og_type: {metadata['og_type']}
---
"""

# Now the actual content
content = """
---
When I was a young 12 year old, I was playing around with Gamemaker.

I took copyright law very seriously, and decided I must make my own music for my games otherwise I would be a criminal.

Thus started my music career, under the moniker Planet Ed (at some point stylised instead as Planet 'ed, a contraction of Planet Head). I didn't know anything about music theory. I had a light background in jazz piano, but I generally couldn't be bothered to learn anything properly.

Destroyed World was the first song I made, I was playing Jak and Daxter at the time so I was inspired by the music in that game. It is unexpectedly a very good piece of music.

The rest of the music may seem weird and disjointed, but I was 12 and to reiterate, I didn't know anything about music theory.

Armed with a copy of Sibelius 3 and an EMU Proteus 2000, I set out to make my first album. I was very proud of it at the time. Now I just look back at it with nostalgia.

I hope you enjoy it.

PS. The recording is terrible because I didn't know how to record properly. I didn't know what a DAW was.
---
title: Blips
file: /assets/music/01_Blips.mp3
title: Heaven
file: /assets/music/02_Heaven.mp3
title: Drum Pie and Peas
file: /assets/music/03_Drum_Pie_and_Peas.mp3
title: Folk Thing
file: /assets/music/04_Folk_Thing.mp3
title: Power Plant [Power Failure Remix]
file: /assets/music/05_Power_Plant_Power_Failure_Remix.mp3
title: Heavy Machinery
file: /assets/music/06_Heavy_Machinery.mp3
title: Aliens
file: /assets/music/07_Aliens.mp3
title: Epic Song
file: /assets/music/08_Epic_Song.mp3
title: Destroyed World
file: /assets/music/09_Destroyed_World.mp3
title: The Banjo Experience
file: /assets/music/10_The_Banjo_Experience.mp3
title: Taking a Ride
file: /assets/music/11_Taking_a_Ride.mp3
title: Power Plant
file: /assets/music/12_Power_Plant.mp3
"""

with open('test-dir/planet_ed.md', 'w') as f:
    f.write(metadata_md + content)

In [58]:
# Alright we gonna add collection metadata to our markdown files
for f in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', f), 'r') as file:
        md = file.read()

    # Add collection metadata
    collection_metadata = "collection: Ed's Blog"
    metadata = md.split('---')[1]
    md = md.replace(metadata, f"{metadata}{collection_metadata}\n")
    # Write it back
    with open(os.path.join('test-dir', 'blogs', f), 'w') as file:
        file.write(md)

# Now same in the music directory
with open('test-dir/planet_ed.md', 'r') as file:
    md = file.read()
metadata = md.split('---')[1]
collection_metadata = "collection: Ed's Music"
# Add collection metadata
md = md.replace(metadata, f"{metadata}{collection_metadata}\n")
# Write it back
with open('test-dir/planet_ed.md', 'w') as file:
    file.write(md)

In [60]:
# Now let's get our feed data and create a list of all our blog posts in reverse chronological order
# We can use this to write to our new firestore collection
feed = db.collection('feed').document('content-log').get().to_dict()
feed

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'}}

In [63]:
new_doc = db.collection('collections').document('eds_blog')
# Let's write a list from our feed data
data = [
    v['location'].split('/')[-1] for v in feed.values()
]
data

['stumbleupon.md',
 'horses.md',
 'charlmes.md',
 'internet.md',
 'oldwebsites.md',
 'hip.md',
 'depressed.md',
 'bananas.md',
 'books.md',
 'dustydrawers.md',
 'onrss.md',
 'prank.md',
 'stress.md',
 'alcoholism.md',
 'burnout.md',
 'discord.md']

In [64]:
# Cool now we can write this to our new collection
new_doc.set({'content': data})

update_time {
  seconds: 1720953587
  nanos: 833201000
}

In [69]:
# We should also add the music album to the music collection
new_doc = db.collection('collections').document('eds_music')
new_doc.set({'content': ['planet_ed.md']})


update_time {
  seconds: 1720957229
  nanos: 499697000
}

In [73]:
# Let's add our music to the feed
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()
data['2005-01-01 00:00:00'] = {
    'location': 'music/planet_ed.md'
}

data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'}}

In [74]:
# Put it back
feed.set(data)

update_time {
  seconds: 1720957492
  nanos: 514354000
}

In [82]:
from random import randint
# Let's try add some comics (Hewligg Urubokkle the old shit comics I made when I was 12)
# Alright we need to go through images and create markdown documents for every page, they don't have names so just call them 'Hewligg Urobokkle 1' etc
for im in os.listdir(os.path.join('STAGING', 'images')):
    filename = im.split(".")[0]

    new_filename = 'hewligg_urobokkle_' + im
    # Rename the image using shutil
    shutil.move(os.path.join('STAGING','images', im), os.path.join('STAGING', 'images', 'hewligg_urobokkle_' + im))
    im = new_filename
    # Make a thumbnail by sampling some of the comic
    img = Image.open(os.path.join('STAGING', 'images', im))
    img = img.convert('RGB')
    # Subsample 256x256 of the page
    width, height = img.size
    max_width = width - 256
    max_height = height - 256
    x0 = randint(0, max_width)
    y0 = randint(0, max_height)
    x1 = x0 + 256
    y1 = y0 + 256
    bounds = (x0, y0, x1, y1)
    cropped_img = img.crop(bounds)
    cropped_img.thumbnail((256,256))

    # Save into the staging area
    cropped_img.save(os.path.join("STAGING", "images", im.split('.')[0] + '_thumbnail.jpg'))
    if '_' in filename:
        filename = " Part ".join(filename.split("_"))
    md = f"""
---
date: YYYY-MM-DD
title: Hewligg Urobokkle {filename}
description: Description
author: Ed
tags: ['Sprite Comic', 'Archive', 'Hewligg Urobokkle']
type: comic
thumbnail: /assets/images/{im.split('.')[0]}_thumbnail.jpg
og_title: Hewligg Urobokkle {filename}
og_description: Description
og_image: /assets/images/{im.split('.')[0]}_thumbnail.jpg
og_type: article
collection: Hewligg Urobokkle
---
# Hewligg Urobokkle {filename}

![Hewligg Urobokkle {filename}](/assets/images/{im})
"""

    with open(os.path.join('STAGING', 'comics', im.split('.')[0] + '.md'), 'w') as f:
        f.write(md)

In [85]:
# Okay so we can now write the metadata for the Hewligg Urobokkle collection
collection = db.collection('collections').document('hewligg_urobokkle')
data = [
    f for f in os.listdir(os.path.join('STAGING', 'comics'))
]
# Sort it based on the number in the title
data = sorted(data, key=lambda x: float('_'.join(x.split('_')[2:]).split('.')[0].replace('_','.')))
data

['hewligg_urobokkle_1.md',
 'hewligg_urobokkle_2.md',
 'hewligg_urobokkle_3.md',
 'hewligg_urobokkle_4.md',
 'hewligg_urobokkle_5.md',
 'hewligg_urobokkle_6.md',
 'hewligg_urobokkle_7.md',
 'hewligg_urobokkle_8.md',
 'hewligg_urobokkle_9.md',
 'hewligg_urobokkle_10.md',
 'hewligg_urobokkle_11.md',
 'hewligg_urobokkle_12.md',
 'hewligg_urobokkle_13.md',
 'hewligg_urobokkle_14.md',
 'hewligg_urobokkle_15.md',
 'hewligg_urobokkle_16.md',
 'hewligg_urobokkle_17.md',
 'hewligg_urobokkle_18.md',
 'hewligg_urobokkle_19.md',
 'hewligg_urobokkle_20_1.md',
 'hewligg_urobokkle_20_2.md',
 'hewligg_urobokkle_20_3.md',
 'hewligg_urobokkle_21_1.md',
 'hewligg_urobokkle_21_2.md',
 'hewligg_urobokkle_21_3.md',
 'hewligg_urobokkle_22_1.md',
 'hewligg_urobokkle_22_2.md',
 'hewligg_urobokkle_23.md',
 'hewligg_urobokkle_24.md',
 'hewligg_urobokkle_25.md',
 'hewligg_urobokkle_26.md',
 'hewligg_urobokkle_27.md',
 'hewligg_urobokkle_28.md',
 'hewligg_urobokkle_29.md',
 'hewligg_urobokkle_30.md',
 'hewligg_uro

In [86]:
# Write it back
collection.set({'content': data})

update_time {
  seconds: 1720964939
  nanos: 861734000
}

In [87]:
# Great now let's add our comments to the feed based on the date of the comic
# But this has slightly more nuance we need a datetime object for the date of the comic
# Some share a date so we need to add a second to the time
feed = db.collection('feed').document('content-log')
feed_data = feed.get().to_dict()
feed_data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'}}

In [91]:
# Now let's create a dict of the comic data
new_feed_data = {}
for comic in data:
    with open(os.path.join('STAGING', 'comics', comic), 'r') as f:
        md = f.read()

    # Get the date
    date = md.split('date: ')[1].split('\n')[0]
    date = dt.strptime(date, "%Y-%m-%d")
    while date.strftime("%Y-%m-%d %H:%M:%S") in new_feed_data:
        date = date + timedelta(seconds=1)

    new_feed_data[date.strftime("%Y-%m-%d %H:%M:%S")] = {
        'location': os.path.join('comics', comic)
    }

assert len(new_feed_data.keys()) == len(data)

In [93]:
# Now extend feed_data
feed_data.update(new_feed_data)
feed_data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '20

In [94]:
# Cool let's put it back
feed.set(feed_data)

update_time {
  seconds: 1720965142
  nanos: 440245000
}

In [97]:
# We need to update the feed again because hewligg_urobokkle_21_3.md said 2024 instead of 2004
feed = db.collection('feed').document('content-log')
feed_data = feed.get().to_dict()
feed_data['2024-08-23 00:00:00']

{'location': 'comics/hewligg_urobokkle_21_3.md'}

In [98]:
# Let's pop that key and reinsert it
old_data = feed_data.pop('2024-08-23 00:00:00')
feed_data['2004-08-23 00:00:00'] = old_data
feed_data


{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_urobokkle_20_3.md'},
 '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon

In [99]:
assert len(feed.get().to_dict()) == len(feed_data)

In [101]:
# Cool let's put it back
feed.set(feed_data)

update_time {
  seconds: 1720965936
  nanos: 530778000
}

In [106]:
# The list of blogs is out of order so let's fix that
blogs_list = []
# Sort feed_data by date - oldest first
feed_data = dict(sorted(feed_data.items(), key=lambda item: item[0]))
for k, v in feed_data.items():
    location = v['location']
    if 'blogs' in location:
        print(k)
        blogs_list.append(location.split('/')[-1])

blogs_list

2023-02-22 00:00:00
2023-04-17 00:00:00
2023-06-05 00:00:00
2023-06-20 14:28:27
2023-07-18 08:52:17
2023-07-23 10:22:07
2023-07-29 09:05:47
2023-08-08 12:02:16
2023-08-22 15:15:32
2023-12-17 16:08:05
2023-12-23 11:03:45
2024-01-04 11:35:56
2024-02-01 17:08:24
2024-03-14 00:00:00
2024-04-23 08:17:04
2024-05-06 09:38:07


['stumbleupon.md',
 'bananas.md',
 'alcoholism.md',
 'oldwebsites.md',
 'onrss.md',
 'burnout.md',
 'dustydrawers.md',
 'stress.md',
 'prank.md',
 'depressed.md',
 'charlmes.md',
 'books.md',
 'hip.md',
 'horses.md',
 'internet.md',
 'discord.md']

In [107]:
# Okay now blogs list looks right let's put it back
new_doc = db.collection('collections').document('eds_blog')
new_doc.set({'content': blogs_list})

update_time {
  seconds: 1720968778
  nanos: 830024000
}

In [116]:
# Okay so currently our comics have a headig but the jinja2 layout deals with the heading so let's strip that out our markdown files
for md_file in os.listdir(os.path.join('CONTENT', 'comics')):
    with open(os.path.join('CONTENT', 'comics', md_file), 'r') as f:
        md = f.read()

    # Split on ---
    nothing, metadata, content = md.split('---')
    reconstructed_content = ''
    for line in content.split('\n'):
        if not line:
            continue
        elif line.startswith('#'):
            continue
        reconstructed_content += line + '\n'

    # Strip newlines
    reconstructed_content = '\n' + reconstructed_content.strip()

    # Join it back together
    md = '---'.join([nothing, metadata, reconstructed_content])

    # Write it back
    with open(os.path.join('CONTENT', 'comics', md_file), 'w') as f:
        f.write(md)


In [117]:
# Now deal with pixelated peculirarities
import json
with open(os.path.join('STAGING', 'comic_data.json'), 'r') as f:
    comic_data = json.loads(f.read())
comic_data

[{'name': 'Sonic the Hedgehog in Chilli Dog Zone',
  'description': "Sonic the Hedgehog tells Commander Keen to have a chilli dog but finds out he doesn't know what they are, so they go to Chilli Dog Zone.",
  'date': '2023-07-03',
  'src': 'comic1.png',
  'tooltip': 'Sonk 3'},
 {'name': 'Gotta Grow Pasta',
  'description': "Knuckles the Echidna tries to punch Sonic but it doesn't work because he's too fast, so he grows pasta.",
  'date': '2023-07-04',
  'src': 'comic2.png',
  'tooltip': 'BBC pasta woman'},
 {'name': "Alex Kidd's Whacky Wheelie Adventure",
  'description': "Alex Kidd drives a car and it's whacky and wheelie.",
  'date': '2023-07-04',
  'src': 'comic3.png',
  'tooltip': 'Read Alex Kidd Buys Heroin'},
 {'name': 'Guybrush Threepwood and the Ordinary Rubber Chicken With a Pulley in the Middle of Th eK',
  'description': "Guybrush Threepwood finds a perfectly normal rurbber chicken with a pulley in the middle. OR DOES HE? (He does.) Oh, and there's a monkey. And a pirate. n

In [121]:
# Okay so we gotta loop over this, generate metadata, generate content, add hover_text, and link up the thumbnail and og tag
feed = {}
for comic in comic_data:
    metadata = {}
    metadata['title'] = comic['name']
    metadata['description'] = comic['description']
    metadata['date'] = comic['date']
    metadata['hover_text'] = comic['tooltip']
    metadata['author'] = 'Ed'
    metadata['tags'] = ['Sprite Comic', 'Irony']
    metadata['type'] = 'comic'
    metadata['og_title'] = comic['name']
    metadata['og_description'] = comic['description']
    metadata['og_type'] = 'article'
    metadata['collection'] = 'Pixelated Peculirarities'

    # Now generate the thumbnail and organise the images
    og_name = 'comic_' + comic['src'].split('comic')[1]
    og_img = os.path.join('STAGING', 'og', og_name)
    new_name = 'pp_'+og_name.split('.')[0] + '_og.jpg'
    shutil.copy(og_img, os.path.join('STAGING', 'images', new_name))
    metadata['og_image'] = f"/assets/images/{new_name}"

    # Thumbnail
    img = Image.open(og_img)
    img.thumbnail((256, 256))
    img = img.convert('RGB')

    thumbnail_name = 'pp_'+og_name.split('.')[0] + '_thumbnail.jpg'
    img.save(os.path.join('STAGING', 'images', thumbnail_name))

    metadata['thumbnail'] = f"/assets/images/{thumbnail_name}"

    # Move the src image to the images directory with a new name
    comic_filename = f'pp_{comic["src"]}'
    shutil.copy(os.path.join('STAGING', 'img', comic['src']), os.path.join('STAGING', 'images', comic_filename))

    # Now finally generate the markdown doc
    content = f"""---
date: {metadata['date']}
title: {metadata['title']}
description: {metadata['description']}
author: {metadata['author']}
tags: {metadata['tags']}
type: {metadata['type']}
thumbnail: {metadata['thumbnail']}
og_title: {metadata['og_title']}
og_description: {metadata['og_description']}
og_image: {metadata['og_image']}
og_type: {metadata['og_type']}
collection: {metadata['collection']}
hover_text: {metadata['hover_text']}
---
![{metadata['title']}](/assets/images/{comic_filename})
"""

    with open(os.path.join('STAGING', 'comics', comic_filename.split('.')[0] + '.md'), 'w') as f:
        f.write(content)

    # Now get the datetime of edit from the file src
    edited_time = dt.fromtimestamp(os.path.getmtime(os.path.join('STAGING', 'img', comic['src'])))

    str_time = edited_time.strftime("%Y-%m-%d %H:%M:%S")

    # Add to feed
    feed[str_time] = {
        'location': os.path.join('comics', comic_filename.split('.')[0] + '.md')
    }

In [122]:
# Finally we need a content feed for the Pixelated Peculirarities collection
new_doc = db.collection('collections').document('pixelated_peculirarities')
data = [
    f for f in os.listdir(os.path.join('STAGING', 'comics')) if not 'thumbnail' in f and not 'og' in f
]
data

['pp_comic12.md',
 'pp_comic17.md',
 'pp_comic8.md',
 'pp_comic11.md',
 'pp_comic5.md',
 'pp_comic4.md',
 'pp_comic10.md',
 'pp_comic6.md',
 'pp_comic9.md',
 'pp_comic7.md',
 'pp_comic3.md',
 'pp_comic16.md',
 'pp_comic2.md',
 'pp_comic15.md',
 'pp_comic13.md',
 'pp_comic1.md',
 'pp_comic14.md']

In [124]:
# This data needs to be sorted by date
data = sorted(data, key=lambda x: int(x.split('comic')[1].split('.')[0]))
data

['pp_comic1.md',
 'pp_comic2.md',
 'pp_comic3.md',
 'pp_comic4.md',
 'pp_comic5.md',
 'pp_comic6.md',
 'pp_comic7.md',
 'pp_comic8.md',
 'pp_comic9.md',
 'pp_comic10.md',
 'pp_comic11.md',
 'pp_comic12.md',
 'pp_comic13.md',
 'pp_comic14.md',
 'pp_comic15.md',
 'pp_comic16.md',
 'pp_comic17.md']

In [125]:
# Write this to the collection
new_doc.set({'content': data})

update_time {
  seconds: 1720974531
  nanos: 597079000
}

In [126]:
feed

{'2023-07-03 21:52:08': {'location': 'comics/pp_comic1.md'},
 '2023-07-04 12:53:11': {'location': 'comics/pp_comic2.md'},
 '2023-07-04 17:10:46': {'location': 'comics/pp_comic3.md'},
 '2023-07-06 07:23:12': {'location': 'comics/pp_comic4.md'},
 '2023-07-06 09:49:26': {'location': 'comics/pp_comic5.md'},
 '2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2023-07-08 15:20:42': {'location': 'comics/pp_comic7.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2023-07-13 10:00:22': {'location': 'comics/pp_comic9.md'},
 '2023-07-18 13:16:07': {'location': 'comics/pp_comic10.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-22 16:33:22': {'location': 'comics/pp_comic12.md'},
 '2023-07-30 08:18:48': {'location': 'comics/pp_comic13.md'},
 '2023-12-11 10:10:20': {'location': 'comics/pp_comic14.md'},
 '2023-12-12 17:11:52': {'location': 'comics/pp_comic15.md'},
 '2023-12-26 14:03:03': {'location': 'comics/pp_comic16.md'},
 '2024-06-07 12:2

In [127]:
# Let's get the feed, update it with this and put it back
old_feed = db.collection('feed').document('content-log')
old_feed = old_feed.get().to_dict()
old_feed


{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_ur

In [128]:
# Update with the new feed
old_feed.update(feed)
old_feed

{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_ur

In [129]:
# Cool let's put it back
feed = db.collection('feed').document('content-log')
feed.set(old_feed)

update_time {
  seconds: 1720974593
  nanos: 988740000
}

In [130]:
# Sick now let's move staging to content
for root, dirs, files in os.walk('STAGING'):
    for file in files:
        shutil.move(os.path.join(root, file), os.path.join('CONTENT', root.split('/')[-1], file))
        print('Moved', file, 'to', os.path.join('CONTENT', root.split('/')[-1], file))

Moved pp_comic4.png to CONTENT/images/pp_comic4.png
Moved pp_comic_13_thumbnail.jpg to CONTENT/images/pp_comic_13_thumbnail.jpg
Moved pp_comic_8_thumbnail.jpg to CONTENT/images/pp_comic_8_thumbnail.jpg
Moved pp_comic_12_thumbnail.jpg to CONTENT/images/pp_comic_12_thumbnail.jpg
Moved pp_comic11.png to CONTENT/images/pp_comic11.png
Moved pp_comic_2_og.jpg to CONTENT/images/pp_comic_2_og.jpg
Moved pp_comic_5_thumbnail.jpg to CONTENT/images/pp_comic_5_thumbnail.jpg
Moved pp_comic14.png to CONTENT/images/pp_comic14.png
Moved pp_comic_13_og.jpg to CONTENT/images/pp_comic_13_og.jpg
Moved pp_comic_11_thumbnail.jpg to CONTENT/images/pp_comic_11_thumbnail.jpg
Moved pp_comic3.png to CONTENT/images/pp_comic3.png
Moved pp_comic16.png to CONTENT/images/pp_comic16.png
Moved pp_comic_1_thumbnail.jpg to CONTENT/images/pp_comic_1_thumbnail.jpg
Moved pp_comic_15_thumbnail.jpg to CONTENT/images/pp_comic_15_thumbnail.jpg
Moved pp_comic_16_thumbnail.jpg to CONTENT/images/pp_comic_16_thumbnail.jpg
Moved pp_c